In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator  
from keras.preprocessing import image
from keras.layers import BatchNormalization
from keras.models import load_model
from keras.models import Model
from keras.utils import plot_model
from IPython.display import Image
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

from PIL import Image
import numpy as np
import PIL

from keras.applications.nasnet import NASNetMobile

import random
random.seed(0)

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:
training_set = train_datagen.flow_from_directory('waste-classification-data/DATASET/TRAIN',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('waste-classification-data/DATASET/TEST',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [4]:
# base_model = NASNetMobile(weights='imagenet', include_top=False, pooling='max')

In [5]:
# tensorboard = TensorBoard(log_dir='log_nas/')
checkpointer = ModelCheckpoint(filepath='tmp/weights.hdf5', verbose=1, save_best_only=True)

In [6]:
# x = base_model.output
# x = Dense(64, activation='relu',kernel_initializer='glorot_normal')(x)
# x = Dropout(0.5)(x)
# x = BatchNormalization()(x)
# x = Dense(32, activation='relu',kernel_initializer='glorot_normal')(x)
# x = Dropout(0.5)(x)
# x = Dense(16, activation='relu')(x)
# predictions = Dense(1,activation='sigmoid')(x)

# model = Model(inputs=base_model.input, outputs=predictions)
# for layer in base_model.layers:
#     layer.trainable = False
# model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
#                    metrics = ['accuracy'])

# Initialising the CNN
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3),
                      activation = 'relu'))

# Step 2 - Pooling
model.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
model.add(Flatten())

# Step 4 - Full connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                   metrics = ['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total para

In [8]:
model.fit_generator(training_set,
                         steps_per_epoch = 500,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = 2000,
                   callbacks=[checkpointer])

Epoch 1/10
500/500 [==============================] - 105s 211ms/step - loss: 0.4535 - acc: 0.7936 - val_loss: 0.3301 - val_acc: 0.8804

Epoch 00001: val_loss improved from inf to 0.33006, saving model to tmp/weights.hdf5
Epoch 2/10
500/500 [==============================] - 95s 190ms/step - loss: 0.3920 - acc: 0.8253 - val_loss: 0.3034 - val_acc: 0.8889

Epoch 00002: val_loss improved from 0.33006 to 0.30339, saving model to tmp/weights.hdf5
Epoch 3/10
500/500 [==============================] - 143s 285ms/step - loss: 0.3735 - acc: 0.8402 - val_loss: 0.2971 - val_acc: 0.8878

Epoch 00003: val_loss improved from 0.30339 to 0.29714, saving model to tmp/weights.hdf5
Epoch 4/10
500/500 [==============================] - 128s 256ms/step - loss: 0.3545 - acc: 0.8477 - val_loss: 0.3138 - val_acc: 0.8783

Epoch 00004: val_loss did not improve from 0.29714
Epoch 5/10
500/500 [==============================] - 122s 244ms/step - loss: 0.3315 - acc: 0.8587 - val_loss: 0.3269 - val_acc: 0.8718

Ep

In [9]:
# model.save('my_model_transfer_nasnet.h5')
model.save('cnn.h5')